# Ranked Programming Tutorial: Boolean-Circuit Storyline

This notebook scaffolds a hands-on path through the library: basic ranks (κ/τ), a Boolean-circuit demo, a Structural Ranking Model (SRM) with interventions, and a light sketch of constraint reasoning.

In [30]:
# Environment bootstrap: add repository 'src' to sys.path so imports work when running in-place
import os, sys
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))  # docs/tutorials -> repo root
src_path = os.path.join(repo_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Imports from the library
from ranked_programming import Ranking, nrm_exc, rlet, rlet_star
from ranked_programming.causal.srm import Variable, StructuralRankingModel

# Optional: show version if installed as a package
try:
    from importlib.metadata import version
    __rp_ver__ = version('ranked_programming')
except Exception:
    __rp_ver__ = 'src checkout'
__rp_ver__

# Helper constants for fault states
No_Fault = True
Is_Faulted = False

## 1. Hello, Ranking Theory (κ/τ)

Ranking Theory models an agent's degrees of surprise (or implausibility) via the negative ranking function κ. Formally, κ maps possible worlds to ℕ∪{∞} and lifts to propositions by the minimality rule: κ(A) = min{κ(w) : w ∈ A}. Intuitively, κ(A)=0 means A isn't disbelieved at all, while higher integers encode stronger disbelief; κ(W)=0 and the law of disjunction κ(A ∪ B) = min(κ(A), κ(B)) ensure coherence. In this library, a Ranking is a lazy generator of (value, rank) pairs that realizes κ operationally. Combinators like nrm_exc(normal, exceptional, penalty) construct tiny κ-structures by yielding the "normal" case at rank 0 and the "exception" at the given penalty; more complex programs compose these pieces while preserving minimal ranks.

To connect disbelief to belief, we use the two-sided ranking τ defined by τ(A) = κ(¬A) − κ(A). Positive τ(A) indicates belief in A, negative indicates disbelief, and τ(A)=0 captures suspension of judgment. The law of negation guarantees at least one of κ(A), κ(¬A) is 0, making τ well-behaved. In code, Ranking.disbelief_rank computes κ(A) as the minimum rank over worlds satisfying A, and Ranking.belief_rank computes τ(A) from κ. Composition operators (e.g., rlet, rlet_star) combine independent pieces by adding ranks and normalizing, mirroring Shenoy's pointwise-addition view of updates. In the circuit below, "worlds" are assignments over N, O1, O2, and OUT; we can query κ/τ for propositions like OUT=True and see how they change under interventions.

We'll start with a tiny distribution over booleans: normally No_Fault, exceptionally Is_Faulted with rank 1. This is our "Hello World".

In [31]:
# A simple Ranking over booleans using nrm_exc(normal, exception, penalty)
r_bool = Ranking(lambda: nrm_exc(No_Fault, Is_Faulted, 1)) 
list(r_bool)  # [(No_Fault, 0), (Is_Faulted, 1)] #Note that 0 is the default rank for the normal case and so is not passed explicitly

[(True, 0), (False, 1)]

The `nrm_exc` (normal_exception) combinator directly realizes Spohn's negative ranking function κ for a binary choice between a "normal" and an "exceptional" outcome. In Spohn's nomenclature, κ assigns ranks to possible worlds: the normal case (e.g., No_Fault) gets κ=0, indicating no disbelief, while the exceptional case (e.g., Is_Faulted) gets κ equal to the penalty parameter, signifying a degree of surprise. This mirrors the law of disjunction, where the rank of the disjunction is the minimum of the individual ranks, ensuring the normal path is preferred unless overridden. Here, the penalty of 1 is a conventional default for simple binary uncertainties—it represents a single unit of disbelief, making the exceptional outcome "just surprising" but not impossibly so. In more complex scenarios, higher penalties (e.g., 2 or 3) can model stronger exceptions, reflecting graded implausibility as per Spohn's theory, where ranks accumulate through composition to capture cumulative disbelief in compound propositions. The rank 0 for the normal case is the default assigned by `nrm_exc` to represent the absence of surprise, analogous to Spohn's κ(W)=0 for the tautology, but here it's a direct implementation choice for the "normal" branch.

## 2. Boolean Circuit (fault diagnosis flavor)

This section introduces a simple Boolean circuit example to demonstrate how rankings propagate uncertainty through logic gates. We'll model a circuit with fixed inputs (i1=False, i2=False, i3=True) and three potential faults: one in a NOT gate (N), and two in OR gates (O1, O2). Each fault is normally absent (No_Fault, meaning the gate works correctly), but can exceptionally be present (Is_Faulted, meaning the gate fails). The circuit computes (NOT i1) OR i2 OR i3, but with faults, the logic may short-circuit to Is_Faulted. We'll use rankings to explore all combinations of faults and their impact on the output.

```
i1 --> [NOT] --> l1 --> [OR] --> l2 --> [OR] --> OUT
        (N)              (O1)              (O2)
i2 ------------------>
i3 --------------------------------------->
```

Here, i1, i2, i3 are inputs, l1 and l2 are intermediate outputs, and OUT is the final output. Faults N, O1, O2 can cause their respective gates to short-circuit to Is_Faulted.

The `rlet` combinator used here implements Spohn's theory by combining independent epistemic states through a cartesian product, summing ranks to reflect cumulative disbelief. Analogous to `nrm_exc` directly realizing the negative ranking function κ for binary choices, `rlet` embodies the combination rule for independent disbelief functions, akin to Shenoy's pointwise addition for scalable belief updates. Its purpose is to model scenarios where multiple independent uncertainties (like faults) interact deterministically, allowing us to compute the joint ranking over all possible outcomes while preserving the graded plausibility from each component.

### Circuit Logic Table

To visualize the circuit, here's a table showing the intermediate and final outputs for each combination of faults (N, O1, O2). Inputs are fixed: i1=False, i2=False, i3=True. "No_Fault" means no fault (gate works correctly), "Is_Faulted" means fault (gate shorts to Is_Faulted, overriding normal logic).

| N       | O1      | O2      | l1 (NOT i1) | l2 (l1 OR i2) | OUT (l2 OR i3) |
|---------|---------|---------|-------------|---------------|----------------|
| No_Fault| No_Fault| No_Fault| True       | True         | True          |
| No_Fault| No_Fault| Is_Faulted| True       | True         | Is_Faulted         |
| No_Fault| Is_Faulted| No_Fault| True       | Is_Faulted        | True          |
| No_Fault| Is_Faulted| Is_Faulted| True       | Is_Faulted        | Is_Faulted         |
| Is_Faulted| No_Fault| No_Fault| Is_Faulted      | Is_Faulted        | True          |
| Is_Faulted| No_Fault| Is_Faulted| Is_Faulted      | Is_Faulted        | Is_Faulted         |
| Is_Faulted| Is_Faulted| No_Fault| Is_Faulted      | Is_Faulted        | True          |
| Is_Faulted| Is_Faulted| Is_Faulted| Is_Faulted      | Is_Faulted        | Is_Faulted         |

This table shows how faults propagate: when a gate has a fault (Is_Faulted), it outputs Is_Faulted regardless of its inputs, effectively short-circuiting the logic. For example, in row 3 (O1=Is_Faulted), l2=Is_Faulted even though l1=True and i2=False, because the OR gate is faulty. Any single fault can still allow OUT=True in some cases (e.g., row 5), but multiple faults force OUT=Is_Faulted. The rankings will assign lower ranks to more plausible (fewer faults) scenarios.

In [ ]:
# Boolean circuit using rlet to combine uncertainty across N, O1, O2

def boolean_circuit():
    # N: Fault in NOT gate (No_Fault = no fault, Is_Faulted = fault present)
    N = Ranking(lambda: nrm_exc(No_Fault, Is_Faulted, 1))
    # O1: Fault in first OR gate (No_Fault = no fault, Is_Faulted = fault present)
    O1 = Ranking(lambda: nrm_exc(No_Fault, Is_Faulted, 1))
    # O2: Fault in second OR gate (No_Fault = no fault, Is_Faulted = fault present)
    O2 = Ranking(lambda: nrm_exc(No_Fault, Is_Faulted, 1))

    def circuit(N, O1, O2):
        # Fixed inputs as in the example: i1=False, i2=False, i3=True
        i1, i2, i3 = False, False, True
        # l1: Output of NOT i1, but if N is Is_Faulted (fault), it shorts to Is_Faulted
        l1 = (not i1) if N else Is_Faulted
        # l2: Output of l1 OR i2, but if O1 is Is_Faulted (fault), it shorts to Is_Faulted
        l2 = (l1 or i2) if O1 else Is_Faulted
        # out: Final output of l2 OR i3, but if O2 is Is_Faulted (fault), it shorts to Is_Faulted
        out = (l2 or i3) if O2 else Is_Faulted
        # Return a dict with faults and output for clarity
        return {"N": N, "O1": O1, "O2": O2, "OUT": out}

    return Ranking(lambda: rlet([('N', N), ('O1', O1), ('O2', O2)],
                                 lambda N, O1, O2: circuit(N, O1, O2)))

rc = boolean_circuit()
# Show distinct outcomes with their minimal ranks
# This code block post-processes the Ranking's output to group unique assignments and ensure minimal ranks,
# without duplicating the core combinator logic (which already provides min ranks per assignment).
from collections import defaultdict
acc = defaultdict(int)
for state, r in rc:
    key = (state['N'], state['O1'], state['O2'], state['OUT'])
    acc[key] = min(acc[key] or r, r)
result = sorted(acc.items(), key=lambda kv: kv[1])

# Print as a table
print("N\tO1\tO2\tOUT\tRank")
for (n, o1, o2, out), r in result:
    print(f"{n}\t{o1}\t{o2}\t{out}\t{r}")

N	O1	O2	OUT	Rank
True	True	True	True	0
True	True	False	False	1
True	False	True	True	1
False	True	True	True	1
True	False	False	False	2
False	True	False	False	2
False	False	True	True	2
False	False	False	False	3


### Interpretation of Ranks

The table above shows the minimal ranks for each possible combination of faults (N, O1, O2) and the resulting circuit output (OUT). In Spohn's Ranking Theory, lower ranks indicate higher plausibility (less disbelief or less surprise) of the outcome, while higher ranks indicate stronger disbelief or more surprise in that scenario.

- **Rank 0**: The most plausible scenario is no faults at all (N=True, O1=True, O2=True), resulting in OUT=True. This aligns with the normal operation of the circuit: (NOT False) OR False OR True = True OR False OR True = True.
- **Rank 1**: Scenarios with exactly one fault. For example:
  - If only O2 is faulted (N=True, O1=True, O2=False), OUT=False because the final OR gate shorts to False.
  - If only N or O1 is faulted, OUT can still be True in some cases, but it's less plausible than no faults.
- **Rank 2**: Scenarios with exactly two faults. These are even less plausible, and OUT is typically False unless the faults don't affect the path to True.
- **Rank 3**: The least plausible scenario is all three faults (N=False, O1=False, O2=False), resulting in OUT=False. This represents the highest disbelief or being the most surprising, as multiple independent faults are unlikely.

The ranks accumulate additively through `rlet`, reflecting the combined disbelief from each fault. This demonstrates how rankings quantify graded uncertainty in fault diagnosis, prioritizing simpler explanations (fewer faults) over complex ones.

## 3. Structural Ranking Model (SRM) + interventions (do)

### Brief Definitions and Purposes
- **Structural Ranking Model (SRM)**: An extension of causal modeling to Ranking Theory, where we represent a system as a Directed Acyclic Graph (DAG) of variables connected by local mechanisms. Each variable has a ranking over its possible values, and mechanisms define how parent variables influence child variables. SRM is designed to model causal relationships probabilistically using rankings, allowing us to compute joint rankings over the entire system and handle uncertainty in a graded, disbelief-based way.
- **Interventions ("do")**: In causal inference, `do(X=x)` represents an external intervention that forces variable X to take value x, overriding its natural mechanism. In SRM, this is implemented as "graph surgery," where we modify the mechanism of X to deterministically yield x at rank 0, ignoring its parents. This allows us to simulate counterfactual scenarios, such as "what if we force a fault?"

### Why Use SRM and "do" in This Example?
SRM and interventions are used here to demonstrate causal reasoning in the Boolean circuit. Instead of just enumerating all fault combinations (as in the previous section), SRM models the circuit as a causal DAG: faults (N, O1, O2) influence intermediate signals (l1, l2), which affect the output (OUT). This captures the structural dependencies and allows us to apply interventions to simulate specific fault scenarios.

### What the Example Shows
The example models the same circuit as before but as an SRM, then applies `do(O2=False)` to intervene on the O2 fault (forcing it to be present). It compares the minimal disbelief ranks (κ) for OUT=True before and after the intervention:
- **Before intervention**: The natural ranking reflects all possible fault combinations, with lower ranks for fewer faults.
- **After intervention**: By forcing O2 to be faulted, we eliminate scenarios where O2 is not faulted, increasing disbelief in OUT=True (higher κ). This illustrates how interventions propagate disbelief through the causal graph, showing the causal effect of a fault on the output. It highlights SRM's ability to handle "what if" questions in uncertain systems, useful for diagnosis or planning.

In [ ]:
# Apply an intervention (graph surgery): do(O2=False)
# srm.do({ 'O2': False }) performs "graph surgery" on the SRM: it overrides the mechanism of O2 to always return False (faulted) at rank 0,
# ignoring its natural dependencies. This simulates forcing the O2 fault to be present.
srm_do = srm.do({ 'O2': False })

# Convert the intervened SRM back to a joint Ranking over all variables.
# This computes the new ranking distribution after the intervention.
joint_do = srm_do.to_ranking()

# Compare minimal ranks of OUT before/after the intervention
# Use the built-in disbelief_rank method instead of a custom function to avoid duplicating core logic.
k_before_true = joint.disbelief_rank(lambda a: a['OUT'] is True)
k_after_true = joint_do.disbelief_rank(lambda a: a['OUT'] is True)
result = (k_before_true, k_after_true)
print(f"Minimal disbelief rank for OUT=True: Before intervention: {k_before_true}, After do(O2=False): {k_after_true}")

# Print table of rankings after intervention (O2 fixed to False)
print("\nRankings after intervention do(O2=False):")
print("N\tO1\tO2\tOUT\tRank")

# This code block post-processes the intervened Ranking's output to group unique assignments and ensure minimal ranks,
# without duplicating the core combinator logic (which already provides min ranks per assignment).
from collections import defaultdict
acc_do = defaultdict(int)
for state, r in joint_do:
    # Collect the minimal rank for each unique (N, O1, O2, OUT) combination
    key = (state['N'], state['O1'], state['O2'], state['OUT'])
    acc_do[key] = min(acc_do[key] or r, r)
result_do = sorted(acc_do.items(), key=lambda kv: kv[1])
for (n, o1, o2, out), r in result_do:
    print(f"{n}\t{o1}\t{o2}\t{out}\t{r}")

result

Minimal disbelief rank for OUT=True: Before intervention: 0, After do(O2=False): inf

Rankings after intervention do(O2=False):
N	O1	O2	OUT	Rank
True	True	False	False	0
True	False	False	False	1
False	True	False	False	1
False	False	False	False	2


(0, inf)

### Interpretation of SRM Intervention Results

The output above demonstrates the effect of intervening on O2 (forcing it to be faulted) in the Structural Ranking Model:

- **Minimal Ranks Comparison**: Before intervention, OUT=True has rank 0 (possible with no faults). After `do(O2=False)`, OUT=True becomes impossible (rank inf), as the O2 fault always shorts the final OR gate to False, overriding any other logic.

- **Post-Intervention Table**: With O2 fixed to False, the table shows the remaining uncertainty over N and O1:
  - **Rank 0**: N=True, O1=True, O2=False, OUT=False. Even with no faults in N/O1, O2's fault forces OUT=False.
  - **Rank 1**: N=True, O1=False, O2=False, OUT=False (one additional fault in O1).
  - **Rank 2**: N=False, O1=True, O2=False, OUT=False (one additional fault in N).
  - **Rank 3**: N=False, O1=False, O2=False, OUT=False (all faults, maximum disbelief).

This illustrates how interventions propagate disbelief through the causal graph: fixing O2 to faulted eliminates scenarios where OUT=True, increasing overall disbelief in that outcome. SRM captures this graded causal reasoning, useful for "what if" analysis in diagnosis or planning.

## 4. Constraint reasoning (c-representations) — sketch
Here's a minimal construction of a constraint network; solving may require Z3 (falls back gracefully).

In [ ]:
from ranked_programming.constraint_reasoning import ConstraintRankingNetwork
net = ConstraintRankingNetwork(variables=['N','O1','O2'])
# Example: declare O1 and O2 as mutually exclusive (type=2)
net.add_constraint('O1','O2', 2)
# Peek at internal graph structure
net._build_constraint_graph()

### Next steps
- Expand identification and causal demos (backdoor/frontdoor).
- Add belief propagation on larger graphs.
- Extend c-inference with concrete conditional bases and skeptical checks.